In [1]:
import os
import cv2

from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [2]:
# Initialise model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
cfg.MODEL.DEVICE = "cpu"
predictor = DefaultPredictor(cfg)

In [4]:
data_dir = "data/scene/"
output_dir = "output/"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
imageList = os.listdir(data_dir)
for image in imageList:
    print("Processing... ",image)
    input_image = cv2.imread(os.path.join(data_dir, image))
    height, width, _ = input_image.shape
        
    panoptic_seg, segments_info = predictor(input_image)["panoptic_seg"]
    v = Visualizer(input_image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
    segmented_image = out.get_image()[:, :, ::-1]
    
    # Resize image if required
    if not segmented_image.shape[:2] == (height, width):
        segmented_image = cv2.resize(segmented_image, (height, width))
    
    # Combine the segmented and orignal image
    combined_image = cv2.hconcat([segmented_image, input_image])
    cv2.imwrite(os.path.join(output_dir, image),combined_image)

Processing...  6178_3003_0.jpg
Processing...  6178_2473_0.jpg
Processing...  6176_1752_0.jpg
Processing...  6180_1734_0.jpg
